In [99]:
import pandas as pd
import numpy as np

In [100]:
#importing datasets
products_df = pd.read_csv('Products.csv',encoding= 'unicode_escape',usecols=['productId','product'],dtype={'productId': 'int32', 'product': 'str'})
rating_df=pd.read_csv('Products_rating.csv',usecols=['userId', 'productId', 'rating'],
    dtype={'userId': 'int32', 'productId': 'int32', 'rating': 'float32'})

In [101]:
products_df.head()

,productId,product
0,101,Mango Pickle Sweet
1,102,Vegetable Pickle
2,103,Carrot Pickle
3,104,Udad Papad
4,105,Sabudana Papad


In [102]:
rating_df.head()

,userId,productId,rating
0,1,101,7.000
1,1,114,7.500
2,1,111,8.000
3,1,202,8.100
4,1,201,7.800


In [103]:
#Merging dataset
df = pd.merge(rating_df,products_df,on='productId')
df.head()

,userId,productId,rating,product
0,1,101,7.000,Mango Pickle Sweet
1,7,101,9.000,Mango Pickle Sweet
2,16,101,8.400,Mango Pickle Sweet
3,1,114,7.500,Lemon Pickle Sour
4,7,114,8.700,Lemon Pickle Sour


In [104]:
#Finding total rating count (Would be used to find poppularity)
combine_product_rating = df.dropna(axis = 0, subset = ['product'])
product_ratingCount = (combine_product_rating.
     groupby(by = ['product'])['rating'].
     count().
     reset_index().
     rename(columns = {'rating': 'totalRatingCount'})
     [['product', 'totalRatingCount']]
    )
product_ratingCount.head()

,product,totalRatingCount
0,AR HandicraftsSheesham Plate Holder Decorative...,4
1,Aafiya handicrafts® Rust Free Metal Plant Stan...,1
2,"Amazon Brand - Vedaka Chana Garlic Papad, 400g",5
3,"Amazon Brand - Vedaka Moong Garlic Papad, 400g",6
4,"Amazon Brand - Vedaka Moong Special Papad, 400g",2


In [105]:
product_ratingCount

,product,totalRatingCount
0,AR HandicraftsSheesham Plate Holder Decorative...,4
1,Aafiya handicrafts® Rust Free Metal Plant Stan...,1
2,"Amazon Brand - Vedaka Chana Garlic Papad, 400g",5
3,"Amazon Brand - Vedaka Moong Garlic Papad, 400g",6
4,"Amazon Brand - Vedaka Moong Special Papad, 400g",2
...,...,...
69,Woolen Handmade Sweater round neck,1
70,Woolen Vest,2
71,Zaveri Pearls Earrings,1
72,srishopify handicrafts Traditional Fabric Hand...,5


In [106]:
rating_with_totalRatingCount = combine_product_rating.merge(product_ratingCount, left_on = 'product', right_on = 'product', how = 'left')
rating_with_totalRatingCount.head()

,userId,productId,rating,product,totalRatingCount
0,1,101,7.000,Mango Pickle Sweet,3
1,7,101,9.000,Mango Pickle Sweet,3
2,16,101,8.400,Mango Pickle Sweet,3
3,1,114,7.500,Lemon Pickle Sour,3
4,7,114,8.700,Lemon Pickle Sour,3


In [107]:
pd.set_option('display.float_format', lambda x: '%.3f' % x)
print(product_ratingCount['totalRatingCount'].describe())

count   74.000
mean     2.311
std      1.344
min      1.000
25%      1.000
50%      2.000
75%      3.000
max      6.000
Name: totalRatingCount, dtype: float64


In [108]:
popularity_threshold = 2
rating_popular_product= rating_with_totalRatingCount.query('totalRatingCount >= @popularity_threshold')
rating_popular_product.head()

,userId,productId,rating,product,totalRatingCount
0,1,101,7.000,Mango Pickle Sweet,3
1,7,101,9.000,Mango Pickle Sweet,3
2,16,101,8.400,Mango Pickle Sweet,3
3,1,114,7.500,Lemon Pickle Sour,3
4,7,114,8.700,Lemon Pickle Sour,3


In [109]:
rating_popular_product

,userId,productId,rating,product,totalRatingCount
0,1,101,7.000,Mango Pickle Sweet,3
1,7,101,9.000,Mango Pickle Sweet,3
2,16,101,8.400,Mango Pickle Sweet,3
3,1,114,7.500,Lemon Pickle Sour,3
4,7,114,8.700,Lemon Pickle Sour,3
...,...,...,...,...,...
161,7,428,8.400,Handicraft Storeroom Acrylic Decorative Rangol...,3
162,8,428,9.000,Handicraft Storeroom Acrylic Decorative Rangol...,3
163,12,428,8.800,Handicraft Storeroom Acrylic Decorative Rangol...,3
165,9,426,9.300,Handicrafts Paradise Door Hanging Blue Painted...,2


In [110]:
rating_popular_product.shape

(145, 5)

In [111]:
product_features_df=rating_popular_product.pivot_table(index='product',columns='userId',values='rating').fillna(0)
product_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
product,,,,,,,,,,,,,,,,,,,,,
AR HandicraftsSheesham Plate Holder Decorative Showpiece,0.000,0.000,0.000,0.000,0.000,9.100,0.000,8.100,0.000,0.000,...,0.000,0.000,9.100,0.000,0.000,0.000,7.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Chana Garlic Papad, 400g",8.500,0.000,0.000,8.100,6.800,0.000,0.000,8.800,0.000,8.100,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Moong Garlic Papad, 400g",8.100,7.600,8.900,0.000,7.500,9.600,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Moong Special Papad, 400g",0.000,0.000,0.000,8.900,0.000,0.000,0.000,0.000,9.800,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Urad Special Papad, 400g",0.000,0.000,0.000,0.000,0.000,8.100,0.000,0.000,9.700,0.000,...,7.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [112]:
from scipy.sparse import csr_matrix
product_features_df_matrix = csr_matrix(product_features_df.values)
from sklearn.neighbors import NearestNeighbors
model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(product_features_df_matrix)

NearestNeighbors(algorithm='brute', metric='cosine')

In [113]:
product_features_df.shape

(48, 21)

In [114]:
query_index = 20
print(query_index)
distances, indices = model_knn.kneighbors(product_features_df.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 5)

20


In [115]:
product_features_df.head()

userId,1,2,3,4,5,6,7,8,9,10,...,12,13,14,15,16,17,18,19,20,21
product,,,,,,,,,,,,,,,,,,,,,
AR HandicraftsSheesham Plate Holder Decorative Showpiece,0.000,0.000,0.000,0.000,0.000,9.100,0.000,8.100,0.000,0.000,...,0.000,0.000,9.100,0.000,0.000,0.000,7.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Chana Garlic Papad, 400g",8.500,0.000,0.000,8.100,6.800,0.000,0.000,8.800,0.000,8.100,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Moong Garlic Papad, 400g",8.100,7.600,8.900,0.000,7.500,9.600,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Moong Special Papad, 400g",0.000,0.000,0.000,8.900,0.000,0.000,0.000,0.000,9.800,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
"Amazon Brand - Vedaka Urad Special Papad, 400g",0.000,0.000,0.000,0.000,0.000,8.100,0.000,0.000,9.700,0.000,...,7.600,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000


In [118]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(product_features_df.index[query_index]))
    else:
        print('{0}: {1}'.format(i, product_features_df.index[indices.flatten()[i]]))

Recommendations for Handicraft-Palace Cotton Abstract Wall Hanging Poster:

1: Lemon Pickle Sweet
2: Vegetable Pickle 
3: Jackfruit Pickle
4: DRAVY HANDICRAFTS Elephant Door Wall Hanging


In [119]:
product_features_df2 = rating_popular_product.pivot_table(index='userId',columns='product',values='rating').fillna(0)

In [120]:
product_features_df2.head()

product,AR HandicraftsSheesham Plate Holder Decorative Showpiece,"Amazon Brand - Vedaka Chana Garlic Papad, 400g","Amazon Brand - Vedaka Moong Garlic Papad, 400g","Amazon Brand - Vedaka Moong Special Papad, 400g","Amazon Brand - Vedaka Urad Special Papad, 400g",Angira Handicrafts Wooden Serving Tray (Set of 2 ) 12 X 8 INCH with Digital Print (Brown),"Anu Appalam Papad Plain Indian Papad,400g - Pack of 2",BANARASI ALOO PAPAD Regular 250gms,"Brand Basket Handicraft Jewellery Box, Wedding Gift Box, Meenakari Wooden Box, Vanity Box.",Cartilage Helix Stainless Steel Eearring,...,Simran Handicrafts Wooden Spectacle Holder,Sparkling Gold Pendant necklace,Traditional Slipper dark brown,Traditional shoes,Udad Papad,Vegetable Pickle,Wooden Bracelet,Woolen Handmade Sweater V neck,Woolen Vest,srishopify handicrafts Traditional Fabric Handbags for Women
userId,,,,,,,,,,,,,,,,,,,,,
1,0.000,8.500,8.100,0.000,0.000,8.900,0.000,0.000,9.700,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.300
2,0.000,0.000,7.600,0.000,0.000,0.000,7.500,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,9.000,0.000,0.000,8.100
3,0.000,0.000,8.900,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,8.300,0.000,9.800,0.000,0.000,0.000,0.000,0.000,0.000,9.300
4,0.000,8.100,0.000,8.900,0.000,0.000,0.000,7.500,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
5,0.000,6.800,7.500,0.000,0.000,0.000,8.200,0.000,0.000,9.100,...,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,9.700


In [121]:
product_features_df2.shape

(21, 48)

In [122]:
X = product_features_df2.values.T
X.shape

(48, 21)

In [123]:
import sklearn
from sklearn.decomposition import TruncatedSVD

In [124]:
SVD = TruncatedSVD(n_components=12, random_state=17)
matrix = SVD.fit_transform(X)
matrix.shape

(48, 12)

In [125]:
import warnings

In [126]:
warnings.filterwarnings("ignore",category = RuntimeWarning)
corr = np.corrcoef(matrix)
corr.shape

(48, 48)

In [127]:
product_names = product_features_df2.columns
product_list = list(product_names)
Handbag = product_list.index("Handicraft-Palace Cotton Abstract Wall Hanging Poster")
print(Handbag)

20


In [128]:
corr_Handbag = corr[Handbag]

In [129]:
list(product_names[(corr_Handbag<1.0) & (corr_Handbag>0.4)])

['DRAVY HANDICRAFTS Elephant Door Wall Hanging',
 'Jackfruit Pickle',
 'Lemon Pickle Sweet',
 'Mens Cartilage stud Earring',
 'Vegetable Pickle ']